In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from ipywidgets import widgets, VBox
from IPython.display import display, clear_output


def load_data(filename, budget=10):
    df = pd.read_csv(filename)
    strength = df['Shear Modulus'].values

    if len(strength) < budget:
        last_value = strength[-1] if len(strength) > 0 else 0
        strength = np.pad(strength, (0, budget - len(strength)), constant_values=last_value)
    return strength

def load_selected_data(selected_direrctory):
    data = []
    for dir_name in os.listdir(selected_direrctory):
        file = os.path.join(selected_direrctory, dir_name)
        if file.endswith('.csv'):
            strength = load_data(file)
            data.append(strength)
    return data

directories = [ "/Users/yannikmeister/Documents/Capstone_project/Meisterchaeller/Results/gpt-3.5-turbo_1732289038_OpenAIcrystalinfo",
                "/Users/yannikmeister/Documents/Capstone_project/Meisterchaeller/Results/gpt-3.5-turbo_perovskiteopenAI",
                "/Users/yannikmeister/Documents/Capstone_project/Meisterchaeller/Results/gpt-3.5-turbo_1732699786_crystalstructure"]


labels = ["Crystal structure context by OpenAI", "Perovskite info by OpenAi", "Space group info by Materials Project"]


plt.figure(figsize=(10, 6))
count = 0
for dir_path in directories:
    data = load_selected_data(dir_path)
    max_length = max(len(arr) for arr in data)
    #max_length = 10
    data = [np.pad(arr, (0, max_length - len(arr)), 'edge') if len(arr) < max_length else arr for arr in data]
    #print(data)
    data = np.array(data)
    #number_of_runs = len(data)
    cumulative_data = []
    for line in data:
        if line is None:
            continue  # Skip if no data for this line
            # Calculate cumulative max for each run
        cumulative_strengths = np.maximum.accumulate(line)
        cumulative_data.append(cumulative_strengths)
    cumulative_data = np.array(cumulative_data)
    average_array = np.mean(cumulative_data, axis=0)

    x = list(range(1, len(average_array) + 1))
    plt.plot(x, average_array, linestyle='-', label=f"{labels[count]}") #, Runs = {number_of_runs}")
    count += 1
    
plt.axhline(y = 143.46, color='r', linestyle='--', label='Desired Target')
plt.xlabel('Number of Development Cycles')
plt.ylabel('Shear Modulus [GPa]')
#plt.title("Compressive strength – SFDL")
#plt.title("Compressive strength – TVDL")
plt.legend()
plt.grid(True)
plt.show()

